In [15]:
# needed librairies
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.metrics import mean_squared_error
import tensorflow as tf

**Loading Data**

In [16]:
data = pd.read_csv('HIGGS_train.csv', dtype={'8': float, '21': float})

# define the headers of the dataset
column_names = ['class_label', 'lepton_pt', 'lepton_eta', 'lepton_phi', 'missing_energy_magnitude', 'missing_energy_phi',
                'jet_1_pt', 'jet_1_eta', 'jet_1_phi', 'jet_1_btag', 'jet_2_pt', 'jet_2_eta', 'jet_2_phi', 'jet_2_btag',
                'jet_3_pt', 'jet_3_eta', 'jet_3_phi', 'jet_3_btag', 'jet_4_pt', 'jet_4_eta', 'jet_4_phi', 'jet_4_btag',
                'm_jj', 'm_jjj', 'm_lv', 'm_jlv', 'm_bb', 'm_wbb', 'm_wwbb']

# assign the headers to the data
data.columns=column_names

cleaned_data = data.copy();

C:\Users\Malak\AppData\Local\Temp\ipykernel_14468\237186723.py:1: DtypeWarning: Columns (8,21) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('HIGGS_train.csv', dtype={'8': float, '21': float})


**Exploring Data**

In [17]:
# check the first few rows of the data
print("The first few rows:\n")
print(data.head())
print("\n")

# retrive the information of the data
print("Data Information:\n")
print(data.info())
print("\n")

# retrive basic statistics about the data
print("Data Statistics:\n")
print(data.describe())
print("\n")

# retrive the shape of the data
print("Data Shape:\n")
print(data.shape)
print("\n")

The first few rows:

   class_label  lepton_pt  lepton_eta  lepton_phi  missing_energy_magnitude   
0          1.0      0.908       0.329     0.35900                     1.500  \
1          1.0      0.799       1.470    -1.64000                     0.454   
2          0.0      1.340      -0.877     0.93600                     1.990   
3          1.0      1.110       0.321     1.52000                     0.883   
4          0.0      1.600      -0.608     0.00707                     1.820   

   missing_energy_phi  jet_1_pt  jet_1_eta jet_1_phi  jet_1_btag  ...   
0              -0.313     1.100     -0.558     -1.59        2.17  ...  \
1               0.426     1.100      1.280      1.38        0.00  ...   
2               0.882     1.790     -1.650    -0.942        0.00  ...   
3              -1.210     0.681     -1.070    -0.922        0.00  ...   
4              -0.112     0.848     -0.566      1.58        2.17  ...   

   jet_4_eta  jet_4_phi  jet_4_btag   m_jj  m_jjj   m_lv  m_jlv  

In [18]:
# the columns 8 and 21 are of type objects: string
# the values of these columns will be checked, 
# looking for unexpected values that lead to having mixed data types

# column 8
print("Column 8: ")
print(cleaned_data['jet_1_phi'].unique())
# result: 
  # 1. float64 numeric values in string objects
  # 2. float64 numeric values in string objects and stored in a string object

# solution:
cleaned_data['jet_1_phi']=pd.to_numeric(cleaned_data['jet_1_phi'],errors='coerce')
print(cleaned_data['jet_1_phi'].unique().size)

# column 21
print("Column 21: ")
print(cleaned_data['jet_4_btag'].unique())
# result:
  # 1. float64 numeric values in string objects instead of float64 type
  # 2. alphabetical values in string objects

#solution:
cleaned_data['jet_4_btag']=pd.to_numeric(cleaned_data['jet_4_btag'], errors='coerce')
print(cleaned_data['jet_4_btag'].unique().size)


Column 8: 
[-1.59 1.38 -0.942 ... '3.50E-01' '-6.59E-03' '"1.01"']
2312
Column 21: 
[0.0 3.1 1.55 '0.00E+00' '3.10E+00' '1.55E+00' 'error' 's']
4


**Data Cleaning**

In [19]:
# remove the training examples with NaN values from the dataset
cleaned_data.dropna(inplace=True)

# testing the realtime

# the cleaned dataset will be saved to a new CSV file
cleaned_data.to_csv('HIGGS_train_cleaned.csv', index=False)

For **regression and decision trees**, we will use the complete set of features (low-level and high-level combined) to take advantage of the manually constructed high-level features.

**Usage of the complete set of features**

**test_size** determines the proportion of the data that will be allocated for the testing set. In this case, test_size=0.2 means that **20%** of the data will be used **for testing**, and the remaining **80%** will be used for **training**.

**random_state** is an optional parameter that sets the random seed used by the random number generator. This **ensures** that the **random splitting of the data is reproducible**, meaning that if you run the same code multiple times with the same random_state value, you will get the same split of data into training and testing sets.

In [20]:
# Split the dataset into training and testing sets
X = cleaned_data.iloc[:, 1:]  # Select all columns except the first one as features
y = cleaned_data.iloc[:, 0]   # Select the first column as the target variable
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


**Linear Regression Model**

In [21]:
# Create a linear regression model and fit it to the training set
model = LinearRegression()
model.fit(X_train, y_train)

# Use the model to make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the model's mean squared error
mse = mean_squared_error(y_test, y_pred)
print('Mean Squared Error: {:.2f}'.format(mse))

Mean Squared Error: 0.22


**Logistic Regression**

In [22]:
# Create a logistic regression model and fit it to the training set
model = LogisticRegression()
model.fit(X_train, y_train)

# Use the model to make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the model's accuracy
accuracy_logistic = accuracy_score(y_test, y_pred)
print('Accuracy: {:.2f}'.format(accuracy_logistic))

Accuracy: 0.64


**Decision Trees**

In [23]:
# Create a decision tree classifier and fit it to the training set
clf = DecisionTreeClassifier(random_state=42)
clf.fit(X_train, y_train)

# Evaluate the classifier's accuracy on the test set
accuracy_decisionTrees = clf.score(X_test, y_test)
print('Accuracy: {:.2f}'.format(accuracy_decisionTrees))

Accuracy: 0.64


**Neural Network**

According to the paper, for the *Higgs boson benchmarks*, we can use a neural network to classify the data by training it on a set of input features and corresponding labels. For the Higgs boson benchmark, we can use *either* the **low-level features**, the **high-level features**, **or the complete set of features** (low-level and high-level combined) as **inputs** to the neural network.

The paper mentions that standard techniques in high-energy physics data analyses include **feed-forward neural networks with a single hidden layer**, which is an example of a *traditional shallow network*.

However, recent advances in deep-learning techniques may lift the limitations of shallow networks by automatically discovering powerful nonlinear feature combinations and providing better discrimination power than current classifiers.

To train a neural network, we need to choose its architecture (number of layers and units per layer), activation function, and other hyperparameters such as learning rate and regularization. The paper provides some details about the hyperparameters used for training deep neural networks on the Higgs boson. Once the network is trained, we can use it to make predictions on new data by feeding the input features through the network and computing the output probabilities.

According to the paper, for training deep neural networks on the Higgs boson, a **five-layer** neural network with **300 hidden units in each layer** was selected, with a **learning rate of 0.05**, and a weight decay coefficient of 1 × 10 −5.

Pre-training, extra hidden units, and additional hidden layers significantly increased training time without noticeably increasing performance.

Hidden units all used the **tanh activation function**. Weights were initialized from a normal distribution with zero mean and standard deviation 0.1 in the first layer, 0.001 in the output layer, and 0.05 in all other hidden layers. Gradient computations were made on mini-batches of size 100.

In [24]:
# load the low-level features
X = cleaned_data.iloc[:, 2:23] # Select columns 2 to 22 as features: the 21 low-level features
y = cleaned_data.iloc[:, 0]    # Select the first column as the target variable


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

**Tanh**, short for hyperbolic tangent, is a commonly used activation function in neural networks. It is a nonlinear function that **maps the input values to a range between -1 and 1**. The tanh function is defined mathematically as:

tanh(x) = (exp(x) - exp(-x)) / (exp(x) + exp(-x))

Like other activation functions, tanh is used to introduce nonlinearity to the output of a neuron in a neural network. It is often used in the hidden layers of neural networks as it allows the network to **capture more complex patterns in the data**.

**Momentum** is a technique used in the *optimization of gradient descent* algorithms to *accelerate the convergence* of the training process.
**Momentum** adds a fraction of the previous update vector to the current update vector, which helps to smooth the optimization trajectory and avoid getting stuck in local minima.

In [28]:
# Define the AUC metric function
def auc(y_true, y_pred):
    return tf.py_function(roc_auc_score, (y_true, y_pred), tf.float64)

In [29]:
# Define the neural network architecture
model = tf.keras.Sequential([
    tf.keras.layers.Dense(300, activation='tanh', input_dim=X_train.shape[1], kernel_initializer=tf.keras.initializers.RandomNormal(mean=0.0, stddev=0.1)),
    tf.keras.layers.Dense(300, activation='tanh', kernel_initializer=tf.keras.initializers.RandomNormal(mean=0.0, stddev=0.05)),
    tf.keras.layers.Dense(300, activation='tanh', kernel_initializer=tf.keras.initializers.RandomNormal(mean=0.0, stddev=0.05)),
    tf.keras.layers.Dense(300, activation='tanh', kernel_initializer=tf.keras.initializers.RandomNormal(mean=0.0, stddev=0.05)),
    tf.keras.layers.Dense(300, activation='tanh', kernel_initializer=tf.keras.initializers.RandomNormal(mean=0.0, stddev=0.05)),
    tf.keras.layers.Dense(1, activation='sigmoid', kernel_initializer=tf.keras.initializers.RandomNormal(mean=0.0, stddev=0.001))
])

# Compile the model with specified hyperparameters
opt = tf.keras.optimizers.SGD(learning_rate=0.05, momentum=0.9)
model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy_score'])

# Train the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), batch_size=100, epochs=200, verbose=1)

# evaluate the model
y_pred = model.predict(X_test)
y_pred = [1 if y >= 0.5 else 0 for y in y_pred]  # convert probabilities to binary predictions

# calculate the accuracy 
auc = accuracy_score(y_test, y_pred)

print('AUC: {:.2f}'.format(auc))


Epoch 1/200
4800/4800 [==============================] - 99s 20ms/step - loss: 0.6737 - auc: 0.6093 - val_loss: 0.6641 - val_auc: 0.6351
Epoch 2/200
4800/4800 [==============================] - 92s 19ms/step - loss: 0.6566 - auc: 0.6495 - val_loss: 0.6525 - val_auc: 0.6533
Epoch 3/200
4800/4800 [==============================] - 101s 21ms/step - loss: 0.6454 - auc: 0.6706 - val_loss: 0.6460 - val_auc: 0.6667
Epoch 4/200
4800/4800 [==============================] - 98s 20ms/step - loss: 0.6385 - auc: 0.6826 - val_loss: 0.6334 - val_auc: 0.6881
Epoch 5/200
4800/4800 [==============================] - 95s 20ms/step - loss: 0.6324 - auc: 0.6925 - val_loss: 0.6314 - val_auc: 0.6931
Epoch 6/200
4800/4800 [==============================] - 95s 20ms/step - loss: 0.6271 - auc: 0.7003 - val_loss: 0.6252 - val_auc: 0.7050
Epoch 7/200
4800/4800 [==============================] - 92s 19ms/step - loss: 0.6221 - auc: 0.7085 - val_loss: 0.6240 - val_auc: 0.7037
Epoch 8/200
4800/4800 [=================

KeyboardInterrupt: 

The decrease in the mean and standard deviation of the weight initialization is to prevent exploding gradients as we move forward towards the output layer during training.

According to the paper, the primary metric for comparison of deep neural networks in high-energy physics is the area under the receiver operating characteristic curve (**AUC**), with **larger** AUC values indicating **higher** **classification** **accuracy** across a range of threshold choices. For example, for the Higgs benchmark, deep neural networks achieved an AUC of 0.88 on both the low-level and complete feature sets when trained with dropout.